In [1]:
!conda env list
!python -V
import sys
print(sys.executable)

# conda environments:
#
base                     /Applications/anaconda3
phd                   *  /Applications/anaconda3/envs/phd

Python 3.10.0
/Applications/anaconda3/envs/phd/bin/python


In [2]:
# LIBRARIES
import numpy as np
import pandas as pd

# OUR OWN LIBRARIES
import utils
import models
import inference_code

## Gibbs sampling <a class="anchor" id="3.3"></a>

In [6]:
m = models.lungModel()
bp = inference_code.beliefPropagation(print_output=0)

# Gibbs inference code
def gibbs_inference_code(observations, n=1000):

    # Random initialisation
    sample = m.sample()
    inflammations = [sample['Inflammation']]
    if 'Bacterial load' not in observations: bacterial_loads = [sample['Bacterial load']]
    fev1s = [sample['FEV1']]
    if 'Wellness' not in observations: wellnesses = [sample['Wellness']]

    for j in np.arange(1,n):
        # sample FEV1 knowing bacterial load
        down_b = get_message_down_from_observation(observations['Bacterial load'] if 'Bacterial load' in observations else bacterial_loads[j-1],
                                                   m.cpt_FEV1_bl)

        sample_fev1 = utils.threeStatesSample(list(down_b.values()), list(down_b.keys()))
        fev1s.append(sample_fev1)

        # sample bl knowing its markov blanket (fev1 and inflammation)
        if 'Bacterial load' not in observations:
            down_i = get_message_down_from_observation(inflammations[j-1], m.cpt_bl_i)
            up_fev1 = get_message_up_from_observation(fev1s[j], m.cpt_FEV1_bl)
            belief_bl = bp.node_message([down_i, up_fev1])

            sample_bl = utils.threeStatesSample(list(belief_bl.values()), list(belief_bl.keys()))
            bacterial_loads.append(sample_bl)

        # sample i knowing bl and wellness
        up_bl = get_message_up_from_observation(observations['Bacterial load'] if 'Bacterial load' in observations else bacterial_loads[j],
                                                m.cpt_bl_i)
        up_w = get_message_up_from_observation(observations['Wellness'] if 'Wellness' in observations else wellnesses[j-1], m.cpt_w_i)
        down_i = bp.message_down(m.marginal_i) # message down from prior
        belief_i = bp.node_message([up_bl, up_w, down_i]) # compute node belief

        sample_i = utils.threeStatesSample(list(belief_i.values()), list(belief_i.keys()))
        inflammations.append(sample_i)

        # sample w knowing i
        if 'Wellness' not in observations:
            down_i = get_message_down_from_observation(inflammations[j], m.cpt_w_i)

            sample_w = utils.nStatesSample(list(down_i.values()), list(down_i.keys()))
            wellnesses.append(sample_w)


    samples = {}
    if 'Inflammation' not in observations: samples['Inflammation'] = inflammations
    if 'Wellness' not in observations: samples['Wellness'] = wellnesses
    if 'Bacterial load' not in observations: samples['Bacterial load'] = bacterial_loads
    if 'FEV1' not in observations: samples['FEV1'] = fev1s

    return samples

def get_message_up_from_observation(obs, cpt):
    point_mass = bp.observe(obs, utils.get_first_level_keys(cpt))
    message_up = bp.factor_message_up(point_mass, cpt)
    return message_up

def get_message_down_from_observation(obs, cpt):
    point_mass = bp.observe(obs, utils.get_second_level_keys(cpt))
    message_down = bp.factor_message_down(point_mass, cpt)
    return message_down

# Data processing
def get_marginal(name, samples):
    """

    :param name: identifier
    :param samples: a list of values
    :return: distinct values count in list, and related proportions (probability estimate)
    """

    marginal = pd.DataFrame(data={name: samples})
    marginal = pd.DataFrame(data={'count': marginal.value_counts()}).reset_index()
    marginal['P(count)'] = marginal['count']/marginal['count'].sum()
    
    return marginal

In [10]:
# Change precision displayed
pd.set_option("display.precision", 8)

In [12]:
# CASE 1: observe medium bacterial load and infer inflammation
observations = {
    # 'Inflammation': 'absent',
    # 'Wellness': 'small',
    'Bacterial load': 'medium'
    # 'FEV1': 'low'
}
samples = gibbs_inference_code(observations, n=1000000)

# Marginal distribution of inflammation
i_marginal = get_marginal('Inflammation', samples['Inflammation'])
i_marginal

,Inflammation,count,P(count)
0,small,555304,0.555304
1,absent,222387,0.222387
2,heavy,222309,0.222309


In [13]:
# CASE 4: observe 1 for wellness and infer fev1
observations = {
    # 'Inflammation': 'absent',
    'Wellness': 1,
    # 'Bacterial load': 'medium'
    # 'FEV1': 'low'
}
samples = gibbs_inference_code(observations, n=1000000)

# Marginal distribution of inflammation
marginal = get_marginal('FEV1', samples['FEV1'])
marginal

,FEV1,count,P(count)
0,medium,378340,0.378340
1,high,353173,0.353173
2,low,268487,0.268487


In [ ]:
# Marginal distribution of bacterial load
b_marginal = get_marginal(B, b)
b_marginal

In [ ]:
plotFigure(i_marginal, I, column=1, title="Marginal distribution for inflammation for "+str(n_high)+" iterations")
plotFigure(i_marginal_fuzzy, I, column=1, title="Marginal distribution for inflammation "+str(n_low)+" iterations")
plotFigure(b_marginal, B, column=1, title="Marginal distribution for inflammation for "+str(n_high)+" iterations")